## **Libraries**

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install jams

In [2]:
import jams

In [3]:
import os

base_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/SynthTab/luthier_pick/part_1"
audio_ext = ".flac"
jams_ext = ".jams"
output_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/SynthTab/npz_files"
os.makedirs(output_dir, exist_ok=True)

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Automatic Guitar Transcription/nootebooks')

from utils.tab_features.builder import build_tab_npz

## **Extraction Tab Features**

In [5]:
from utils.tab_features.builder import build_tab_npz
from utils.audio_features import CQT, STFT, HCQT, MelSpec, SignalPower, FeatureCombo
import os

pitch_folder = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/SynthTab/luthier_pick/part_1/1 God - Grace - gp4__1 - Acoustic Nylon Guitar__midi"
audio_path = os.path.join(pitch_folder, "luthier_pick_nonoise_mono_body.flac")
jams_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/SynthTab/all_jams/outall/1 God - Grace - gp4__1 - Acoustic Nylon Guitar__midi/1 - Acoustic Nylon Guitar.jams"
output_path = "/content/1_Acoustic_Nylon.npz"

combo = FeatureCombo([
    CQT(sample_rate=22050, hop_length=512, bins_per_octave=24, n_octaves=4, center=True),
    MelSpec(sample_rate=22050, hop_length=512, n_mels=128, center=True),
    STFT(sample_rate=22050, hop_length=512, n_fft=1024, center=True),
    SignalPower(sample_rate=22050, hop_length=512, center=True),
    HCQT(sample_rate=22050, hop_length=512, bins_per_octave=24, n_octaves=4,
         fmin='E2', harmonics=[0.5, 1, 2, 3, 4, 5], center=True)
])

data = build_tab_npz(
    jams_path=jams_path,
    audio_path=audio_path,
    pitch_folder=pitch_folder,
    extracted_features=combo,
    sr=22050,
    hop_length=512,
    fmin=82.41,
    n_bins=192,
    bins_per_octave=24,
    num_harmonics=44,
    max_fret=20,
    silence_val=-1,
    save_path=output_path,
    verbose=True
)

Resampling audio from 44100 Hz to 22050 Hz.
[START] cqt
[OK] cqt: shape = (1, 96, 8743)
[START] melspec
[OK] melspec: shape = (1, 128, 8743)
[START] stft
[OK] stft: shape = (1, 513, 8743)
[START] signalpower
[OK] signalpower: shape = (1, 1, 8743)
[START] hcqt
[OK] hcqt: shape = (1, 6, 96, 8743)
[INFO] Final extracted features: ['cqt', 'melspec', 'stft', 'signalpower', 'hcqt']
Running sanity checks...
[AUDIO CHECK] /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/SynthTab/luthier_pick/part_1/1 God - Grace - gp4__1 - Acoustic Nylon Guitar__midi/luthier_pick_nonoise_mono_body.flac
  - Sample rate mismatch: 44100 (expected: 22050)
[WARN] Could not extract time dimension from pitch_list: 'dict' object has no attribute 'shape'

🧪 Time dimensions (T):
  - tablature: 8743
  - tablature_adj: 8743
  - tablature_rel: 8743
  - onsets: 8743
  - offsets: 8743
  - multi_pitch: 8743
Sanity checks completed.



In [6]:
import numpy as np

data = np.load(output_path, allow_pickle=True)

for key in data:
    print(f"{key}: shape = {data[key].shape}, dtype = {data[key].dtype}")

track: shape = (), dtype = <U25
fs: shape = (), dtype = int64
audio: shape = (4476150,), dtype = float64
tablature: shape = (6, 8743), dtype = int32
tablature_adj: shape = (120, 8743), dtype = float32
tablature_rel: shape = (120, 8743), dtype = float32
tablature_adj_spatial: shape = (6, 21, 8743), dtype = float32
tablature_rel_spatial: shape = (6, 21, 8743), dtype = float32
onsets: shape = (6, 8743), dtype = float32
offsets: shape = (6, 8743), dtype = float32
onsets_h: shape = (6, 44, 8743), dtype = float32
offsets_h: shape = (6, 44, 8743), dtype = float32
multi_pitch: shape = (6, 44, 8743), dtype = float32
notes: shape = (), dtype = object
pitch_list: shape = (), dtype = object
pitch_bins: shape = (192,), dtype = float64
cqt: shape = (1, 96, 8743), dtype = float64
melspec: shape = (1, 128, 8743), dtype = float64
stft: shape = (1, 513, 8743), dtype = float64
signalpower: shape = (1, 1, 8743), dtype = float32
hcqt: shape = (1, 6, 96, 8743), dtype = float64
allow_pickle: shape = (), dtyp

In [7]:
tablature = data["tablature"]  # (6, T) integer matrix

print("Max fret in tablature:", np.max(tablature))
print("Min value in tablature:", np.min(tablature))

Max fret in tablature: 23
Min value in tablature: -1


In [14]:
import pandas as pd
import numpy as np

tablature = data["tablature"]  # (6, T) numpy array

num_strings, num_frames = tablature.shape
silence_val = -1

# Tablature içindeki tüm frete ve silence değerlerine bakarak sınıf sayısını bul
min_fret = np.min(tablature)
max_fret = np.max(tablature)

# Sınıf listesi: sessizlik + tüm frets (örnek: [-1, 0, 1, ..., max_fret])
classes = list(range(min_fret, max_fret + 1))  # genellikle min_fret = -1 (silence)

# Her string için sınıf sayısını hesapla
distribution = {}

for s in range(num_strings):
    counts = pd.Series(tablature[s]).value_counts().sort_index()
    total = counts.sum()
    # counts dizisinde sınıf olmayabilir, eksik sınıf varsa 0 ile tamamla
    counts = counts.reindex(classes, fill_value=0)
    distribution[f"String {s+1}"] = pd.DataFrame({
        "Count": counts,
        "Ratio": counts / total
    })

# Sınıf isimleri
index_names = [f"Class {c}" if c != silence_val else f"Class {silence_val} (silence)" for c in classes]

# Tabloları birleştir
df_distribution = pd.concat(distribution.values(), axis=1)
df_distribution.index = index_names

# Göster
display(df_distribution)

,Count,Ratio,Count,Ratio,Count,Ratio,Count,Ratio,Count,Ratio,Count,Ratio
Class -1 (silence),8686,0.993480,8522,0.974723,8405,0.961341,8640,0.988219,8727,0.998170,8743,1.0
Class 0,0,0.000000,0,0.000000,0,0.000000,5,0.000572,0,0.000000,0,0.0
Class 1,0,0.000000,0,0.000000,0,0.000000,12,0.001373,5,0.000572,0,0.0
Class 2,0,0.000000,0,0.000000,1,0.000114,43,0.004918,3,0.000343,0,0.0
Class 3,0,0.000000,0,0.000000,1,0.000114,10,0.001144,6,0.000686,0,0.0
Class 4,0,0.000000,0,0.000000,13,0.001487,9,0.001029,0,0.000000,0,0.0
Class 5,0,0.000000,6,0.000686,31,0.003546,8,0.000915,0,0.000000,0,0.0
Class 6,0,0.000000,1,0.000114,16,0.001830,10,0.001144,0,0.000000,0,0.0
Class 7,0,0.000000,0,0.000000,42,0.004804,3,0.000343,2,0.000229,0,0.0
Class 8,0,0.000000,7,0.000801,36,0.004118,3,0.000343,0,0.000000,0,0.0


In [10]:
notes = data.get("notes", None)
pitch_list = data.get("pitch_list", None)

if notes is not None:
    for s in range(6):
        sample_notes = notes[s][:5] if len(notes[s]) >= 5 else notes[s]
        print(f"String {s} notes sample:", sample_notes)
else:
    print("notes verisi bulunamadı.")

if pitch_list is not None:
    for s in range(6):
        sample_pitch_list = pitch_list[s][:5] if len(pitch_list[s]) >= 5 else pitch_list[s]
        print(f"String {s} pitch_list sample:", sample_pitch_list)
else:
    print("pitch_list verisi bulunamadı.")

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [11]:
print("notes tipi:", type(notes))
print("notes np.ndarray mı?:", isinstance(notes, np.ndarray))
print("notes shape:", None if not hasattr(notes, 'shape') else notes.shape)
print("notes içeriği örnek:", notes)


notes tipi: <class 'numpy.ndarray'>
notes np.ndarray mı?: True
notes shape: ()
notes içeriği örnek: {0: [(274.9741644639493, (1, 2)), (288.5673090696211, (2, 3)), (313.62717087719784, (3, 4)), (274.9741644639493, (4, 5)), (288.5673090696211, (5, 6)), (313.62550492685364, (6, 7)), (274.9741644639493, (7, 8)), (288.56054320362915, (8, 9)), (313.62717087719784, (9, 10)), (274.9741644639493, (10, 11)), (288.55860713084775, (11, 12)), (313.62550492685364, (12, 13)), (357.59555113618086, (13, 14)), (413.76912482292795, (14, 15)), (288.5550121537895, (15, 16)), (343.7575301941055, (16, 17)), (343.7575301941055, (17, 18)), (349.43929375554006, (18, 19)), (357.59555113618086, (19, 20)), (413.7691248320206, (20, 21)), (288.5550121537895, (21, 22)), (343.7575301941055, (22, 23)), (343.7575301941055, (23, 24)), (349.34654727496445, (24, 25)), (274.9741644639493, (25, 26)), (288.5550121537895, (26, 27)), (297.0069933110353, (27, 28)), (274.9741644639493, (28, 29)), (274.7598139627972, (29, 30)), (3

In [12]:
pitch_list_obj = pitch_list.item()
for s in range(6):
    sample_pitch_list = pitch_list_obj.get(s, [])[:5]
    print(f"String {s} pitch_list sample:", sample_pitch_list)


String 0 pitch_list sample: [274.9741644639493, 288.5673090696211, 313.62717087719784, 274.9741644639493, 288.5673090696211]
String 1 pitch_list sample: [270.7447250179471, 258.3365835649866, 336.7021868753973, 338.46266380666833, 270.7447250179471]
String 2 pitch_list sample: [191.00592840869814, 226.64132847620874, 256.28517788453104, 332.0755768399661, 230.79512478151085]
String 3 pitch_list sample: [242.8138821403379, 177.51171176863568, 242.78720240210387, 183.4007882262524, 257.32331154449383]
String 4 pitch_list sample: [176.83318826050987, 234.528049059821, 175.69540131278654, 179.89605839902327, 175.69445020946014]
String 5 pitch_list sample: [113.56533020564058, 124.93451992919373, 113.46786620222359, 124.94121731921943, 115.80523034461343]


In [14]:
import numpy as np

# Dosyayı yükle
data = np.load("/content/1_Acoustic_Nylon.npz", allow_pickle=True)

# notes ve pitch_list objelerini al
notes = data["notes"].item()
pitch_list = data["pitch_list"].item()

# Her string için notaları yazdır
print("🎵 NOTES")
for s in range(6):
    print(f"String {s}: {len(notes[s])} notes")
    for i, (freq, (start, end)) in enumerate(notes[s][:5]):  # İlk 5 örnek
        print(f"  {i}: freq={freq:.2f} Hz, frames=({start}, {end})")
    if len(notes[s]) > 5:
        print("  ...")

# Her string için pitch_list'i yazdır
print("\n📊 PITCH_LIST")
for s in range(6):
    freqs = pitch_list[s]
    print(f"String {s}:")
    print(f"  Length: {len(freqs)}, first 5 = {freqs[:5]}")

🎵 NOTES
String 0: 111 notes
  0: freq=274.97 Hz, frames=(1, 2)
  1: freq=288.57 Hz, frames=(2, 3)
  2: freq=313.63 Hz, frames=(3, 4)
  3: freq=274.97 Hz, frames=(4, 5)
  4: freq=288.57 Hz, frames=(5, 6)
  ...
String 1: 230 notes
  0: freq=270.74 Hz, frames=(1, 2)
  1: freq=258.34 Hz, frames=(2, 3)
  2: freq=336.70 Hz, frames=(3, 4)
  3: freq=338.46 Hz, frames=(4, 5)
  4: freq=270.74 Hz, frames=(5, 6)
  ...
String 2: 344 notes
  0: freq=191.01 Hz, frames=(1, 2)
  1: freq=226.64 Hz, frames=(2, 3)
  2: freq=256.29 Hz, frames=(3, 4)
  3: freq=332.08 Hz, frames=(4, 5)
  4: freq=230.80 Hz, frames=(5, 6)
  ...
String 3: 139 notes
  0: freq=242.81 Hz, frames=(1, 2)
  1: freq=177.51 Hz, frames=(2, 3)
  2: freq=242.79 Hz, frames=(3, 4)
  3: freq=183.40 Hz, frames=(4, 5)
  4: freq=257.32 Hz, frames=(5, 6)
  ...
String 4: 197 notes
  0: freq=176.83 Hz, frames=(1, 2)
  1: freq=234.53 Hz, frames=(2, 3)
  2: freq=175.70 Hz, frames=(3, 4)
  3: freq=179.90 Hz, frames=(4, 5)
  4: freq=175.69 Hz, frames=

In [9]:
import os
import pickle

pitch_folder = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/SynthTab/luthier_pick/part_1/1 God - Grace - gp4__1 - Acoustic Nylon Guitar__midi"

for s in range(6):
    pitch_path = os.path.join(pitch_folder, f"luthier_pick_nonoise_mono_body_string_{s+1}_pitch.pkl")
    with open(pitch_path, "rb") as f:
        data = pickle.load(f)

    print(f"\n📌 String {s} — File: {os.path.basename(pitch_path)}")
    print(f"  🔢 Type: {type(data)}")
    print(f"  📏 Total Notes: {len(data)}")
    if len(data) > 0:
        print(f"  🎵 First 3 notes: {data[:3]}")



📌 String 0 — File: luthier_pick_nonoise_mono_body_string_1_pitch.pkl
  🔢 Type: <class 'list'>
  📏 Total Notes: 112
  🎵 First 3 notes: [array([0.]), array([251.63945475, 420.14108622, 206.1512117 ,  98.66107335,
        88.27984587,  88.10722056, 110.05975305, 440.56978383,
       440.19228967, 440.63828485, 440.27580525]), array([250.00432149, 419.41861925, 206.15738392,  98.66250427,
        88.27991475,  88.10969418, 110.05974803, 440.57520822,
       440.18516506, 440.639172  , 440.26622241, 440.44975524])]

📌 String 1 — File: luthier_pick_nonoise_mono_body_string_2_pitch.pkl
  🔢 Type: <class 'list'>
  📏 Total Notes: 231
  🎵 First 3 notes: [array([0.]), array([252.08583789, 337.67752747, 167.2087824 ,  66.56725072,
       166.22023899, 331.53727606, 331.73240609, 331.5844965 ,
       331.4814398 , 331.05369339, 331.04302589]), array([171.82995213, 352.90621019, 165.94715477, 166.15185586,
       165.96544857, 165.30589733, 331.49672057, 330.64500576,
       330.55482544, 330.801324

In [17]:
test_jam = jams.load(jams_path, validate = False)

for annotation in test_jam.annotations:
    print(annotation.namespace)
    print(annotation.time, annotation.duration)
    print(annotation.data)

note_tab
0 None
SortedKeyList([Observation(time=2400.0, duration=240.0, value={'fret': 5, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=3840.0, duration=240.0, value={'fret': 5, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=5280.0, duration=240.0, value={'fret': 7, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=10080.0, duration=240.0, value={'fret': 5, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=11520.0, duration=240.0, value={'fret': 5, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=12960.0, duration=240.0, value={'fret': 7, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=17760.0, duration=240.0, value={'fret': 5, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=19200.0, duration=240.0, value={'fret': 5, 'velocity': 111, 'let_ring': True}, confidence=None), Observation(time=20640.0, duration=240.0, value={'fret': 7, 

In [ ]:
from glob import glob
from tqdm import tqdm

jams_paths = sorted(glob(os.path.join(base_dir, "*.jams")))

for jam_path in tqdm(jams_paths):
    audio_path = jam_path.replace(".jams", ".flac")
    track_name = os.path.basename(jam_path).replace(".jams", "")
    save_path = os.path.join(output_dir, f"{track_name}.npz")

    try:
        build_tab_npz(
            jams_path=jam_path,
            audio_path=audio_path,
            extracted_features=None,
            sr=22050,
            hop_length=512,
            save_path=save_path
        )
    except Exception as e:
        print(f"⚠️ Hata: {track_name} — {str(e)}")


## **Extraction Audio Features**